In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine
from sklearn.cluster import KMeans
import scipy.stats as stats
import folium
import itertools
import statistics
import string

import warnings
warnings.filterwarnings("ignore")

# Naver Map API

In [2]:
# *-- Geocoding 활용 코드 --*
import json
import urllib
from urllib.request import Request, urlopen

In [3]:
# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')

In [4]:
# *-- Directions 5 활용 코드 --*

# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)]
# trafast 실시간 빠른길
# tracomfort 실시간 편한길
# traoptimal 실시간 최적
# traavoidtoll 무료 우선
# traavoidcaronly 자동차 전용도로 회피 우선
option = 'traoptimal'

def get_optimal_route(start, goal, option=option ) :

    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')

In [5]:
# 주소에서 위도, 경도 뽑고 dataframe 만드는 함수
def geo_df_coding(addresses):
    geo_list = []
    for address in addresses:
        geo_list.append(get_location(address))
        
    geo_df = pd.DataFrame(geo_list)
    geo_df.columns = ['longitude', 'latitude']
    geo_df = geo_df[['latitude', 'longitude']]
    
    return geo_df, geo_list

In [6]:
# 각 점 간도로상 거리, 시간 계산해주는 함수
def dist_time_mat(geo_df, geo_list):
    dist_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))
    time_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))

    option = 'traoptimal'
    for i in range(geo_df.shape[0]):
        for j in range(geo_df.shape[0]):
            if (i==j):
                dist_mat[i][j] = 0
                time_mat[i][j] = 0
            else:
                results = get_optimal_route(start = geo_list[i], goal = geo_list[j], option=option)

                dist_mat[i][j] = results['route']['traoptimal'][0]['summary']["distance"]
                time_mat[i][j] = results['route']['traoptimal'][0]['summary']["duration"]
                
    return dist_mat, time_mat

In [7]:
# 출발지로부터 각 점에 대한 거리, 시간 계산
def dist_time_start(start, geo_list):
    start_loc = get_location(start)
    
    dist_list = []
    time_list = []

    option = 'traoptimal'

    for i in range(len(geo_list)):
        results = get_optimal_route(start = start_loc, goal = geo_list[i], option=option)

        dist_list.append(results['route']['traoptimal'][0]['summary']["distance"])
        time_list.append(results['route']['traoptimal'][0]['summary']["duration"])
    
    return dist_list, time_list

# Algorithm

- 출발지로 다시 돌아오는 거리 기준 알고리즘

In [8]:
# k-means 적용
def k_means(geo_df, k):
    geo_df_k = geo_df.copy()
        
    geo_df_k = geo_df_k.astype('float')
    
    model = KMeans(n_clusters=k, init='k-means++')
    model.fit(geo_df)
    labels = model.predict(geo_df.values)
    geo_df_k['labels'] = labels
    
    return geo_df_k

In [9]:
# 각 그룹에서 출발점으로부터 모든 정점을 거치고 출발점으로 돌아왔을 때 최단거리 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리
    '''
    
    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
    
    # 모든 경우 거리 합 계산 후 최소값 산출
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum += dist_list_start[permutations[i][-1]]
        
        dist_sum_lst.append(dist_sum)

    # 가장 작은 값 -> 최단거리
    return np.min(dist_sum_lst)

In [10]:
# 각 그룹에서 출발점으로부터 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start_order(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서
    '''

    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
    
    # 모든 경우 거리 합 계산 후 최소값 산출
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum += dist_list_start[permutations[i][-1]]
        
        dist_sum_lst.append(dist_sum)

    # 가장 작은 값 -> 최단거리순서
    return permutations[np.argmin(dist_sum_lst)]

In [11]:
# 각 군집의 출발점으로부터 최단거리끼리 차이의 분산
def start_var_diff(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df, dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 각 군집의 출발점으로부터 최단거리 모든 조합 차이의 분산
    '''
    min_distance = []
    for i in geo_df_k['labels'].unique():
        min_distance.append(min_dist_start(geo_df_k, i, dist_list_start, dist_mat))

    # k combination 2 : 2개씩 차이의 분산
    combinations = list(itertools.combinations(geo_df_k['labels'].unique(), 2))
    
    diff_list = []
    for i, j in combinations:
        diff = np.abs(min_distance[i] - min_distance[j])
        diff_list.append(float(diff))

    return statistics.variance(diff_list)

In [12]:
# 다른 군집 point 중 해당 군집에 가장 가까운 point를 해당 군집으로 포함시키는 함수
# 중심점에서 가장 가까운 point 해당 군집으로 포함시킴
def near_point_start(geo_df_k, label):
    '''
    Input : df, 변경할 군집
    Return : 군집의 중심에서 다른 군집 point 중 가장 가까운 point 해당 군집으로 변경
    '''
    # 원본 dataframe 변경 방지
    geo_df_k_c = geo_df_k.copy()
    
    center = geo_df_k_c.groupby('labels')[['latitude', 'longitude']].mean().reset_index('labels')[['latitude', 'longitude']]

    # 군집이 i가 아닌 포인트 중 가장 가까운 포인트의 군집(labels)을 i로 변경
    idx = geo_df_k_c[geo_df_k_c['labels']!=label].index

    geo_df_k_c.loc[:, "distance_other_center"] = np.NAN
    for j in idx:
        geo_df_k_c.loc[j, "distance_other_center"] = haversine(geo_df_k_c.loc[j, ['latitude', 'longitude']], center.loc[label,['latitude', 'longitude']])
        
    if (geo_df_k_c['labels'].value_counts()[geo_df_k_c.loc[np.argmin(geo_df_k_c['distance_other_center']), 'labels']] != 1):
        geo_df_k_c.loc[np.argmin(geo_df_k_c['distance_other_center']), 'labels'] = label
        
    del geo_df_k_c['distance_other_center']
    
    return geo_df_k_c

In [13]:
# 모든 군집에 대해 point 추가 진행해보고 가장 분산이 작아지는 경우 선택하고 다음 loop 실행
def min_var_start_algorithm(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df
    Return : print - 변경될 떄 마다 cnt, var, return - 최종 df
    '''
    geo_df_k_c = geo_df_k.copy()

    cnt = 0
    # print("초기 Var : {}\n".format(start_var_diff(geo_df_k_c)))
    
    k = len(geo_df_k_c['labels'].unique())

    # 기존 군집화, 각 군집 변화시켜보고 var 비교 후 결정
    while(True):
        var_diff_list = []
        var_diff_list.append(start_var_diff(geo_df_k_c, dist_list_start, dist_mat))
        print("Var : {:.5}\n".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
        
        for label in range(len(geo_df_k_c['labels'].unique())):
            geo_df_k_1 = near_point_start(geo_df_k_c, label)
            var_diff_list.append(start_var_diff(geo_df_k_1, dist_list_start, dist_mat))

        if (np.min(var_diff_list) == var_diff_list[0]):
            break
        else:
            if (k != len(near_point_start(geo_df_k_c, np.argmin(var_diff_list)-1)['labels'].unique())):
                break
            else:
                geo_df_k_c = near_point_start(geo_df_k_c, np.argmin(var_diff_list)-1)
                cnt += 1
                print("변경 횟수 : {}".format(cnt))
                # print("Var : {}\n".format(start_var_diff(geo_df_k_c)))
            
    print("총 변경 횟수 : {}".format(cnt))
    print("최종 결과 Var : {:.5}".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
    return geo_df_k_c

# Main

In [14]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울특별시 도봉구 마들로 656']

# 출발지 : 도봉산역
start = "서울특별시 도봉구 도봉로 964-40"
start_loc = get_location(start)

geo_df, geo_list = geo_df_coding(addresses)

# 출발지로부터 각 점까지 거리 & 시간
dist_list_start, time_list_start = dist_time_start(start, geo_list)

# 각 점 서로 거리 & 시간
dist_mat, time_mat = dist_time_mat(geo_df, geo_list)

In [15]:
pd.DataFrame(dist_mat)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,3396.0,1487.0,576.0,916.0,3666.0,2738.0,3196.0,2908.0,4529.0,1892.0
1,3383.0,0.0,2495.0,3006.0,2334.0,441.0,1772.0,2739.0,2519.0,6717.0,4055.0
2,1595.0,1935.0,0.0,1218.0,546.0,2187.0,397.0,2146.0,1858.0,4929.0,2267.0
3,1062.0,2898.0,989.0,0.0,967.0,3150.0,2240.0,2698.0,2410.0,4165.0,1528.0
4,1164.0,2389.0,480.0,678.0,0.0,2641.0,1741.0,2199.0,1911.0,4949.0,2287.0
5,3525.0,142.0,2637.0,3148.0,2476.0,0.0,1914.0,2881.0,2661.0,8467.0,4966.0
6,1767.0,1827.0,1595.0,1390.0,718.0,2079.0,0.0,2038.0,1750.0,5101.0,2439.0
7,2737.0,3204.0,2096.0,2854.0,2182.0,2503.0,2031.0,0.0,288.0,6565.0,3903.0
8,2577.0,2148.0,1808.0,2566.0,1894.0,2283.0,1743.0,288.0,0.0,6277.0,3615.0
9,3625.0,6090.0,4181.0,3557.0,4159.0,6342.0,5432.0,5890.0,5602.0,0.0,3015.0


In [16]:
pd.DataFrame(time_mat)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,646673.0,419092.0,172745.0,243037.0,748527.0,854421.0,642543.0,642820.0,852926.0,580061.0
1,725644.0,0.0,626235.0,640346.0,596317.0,87296.0,403616.0,621510.0,607946.0,1297481.0,932702.0
2,468865.0,403016.0,0.0,414694.0,170646.0,437804.0,100832.0,527441.0,452004.0,941656.0,620071.0
3,313083.0,689217.0,259434.0,0.0,217962.0,576309.0,714880.0,587276.0,511839.0,792211.0,503289.0
4,352303.0,587611.0,171411.0,223031.0,0.0,622399.0,641197.0,527153.0,451716.0,988526.0,647863.0
5,747490.0,35906.0,660129.0,667189.0,632695.0,0.0,438067.0,657417.0,641291.0,1393267.0,971270.0
6,554139.0,375646.0,358112.0,484993.0,254538.0,410434.0,0.0,502082.0,426645.0,989591.0,692471.0
7,750041.0,606779.0,515283.0,813415.0,554281.0,739311.0,540023.0,0.0,128618.0,1295507.0,924221.0
8,654031.0,530777.0,406719.0,704851.0,445717.0,662860.0,433413.0,74641.0,0.0,1220834.0,839669.0
9,645790.0,1181854.0,817727.0,684643.0,783888.0,1223008.0,1202814.0,1134295.0,1100901.0,0.0,550060.0


In [17]:
print(dist_list_start)
print(time_list_start)

[3384, 6925, 3940, 3316, 3918, 7177, 6018, 5649, 5361, 1077, 2743]
[613982, 1193871, 803910, 669363, 759817, 1253799, 1162694, 1119182, 1091250, 271934, 687604]


In [18]:
# k_means clustering 적용
geo_df_2 = k_means(geo_df, 2)
geo_df_3 = k_means(geo_df, 3)
geo_df_4 = k_means(geo_df, 4)
geo_df_5 = k_means(geo_df, 5)

geo_df_3_start = geo_df_3.copy()
geo_df_4_start = geo_df_4.copy()

In [19]:
# 각 군집마다 최단거리 산출
print("k=3일 때 클러스터 0의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 0, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 1의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 1, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 2의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 2, dist_list_start, dist_mat)))

k=3일 때 클러스터 0의 최단거리 : 10838.0m
k=3일 때 클러스터 1의 최단거리 : 2154m
k=3일 때 클러스터 2의 최단거리 : 15219.0m


In [20]:
# 각 군집마다 최단거리일 때 방문 순서
print("k=3일 때 클러스터 0의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 0, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 1의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 1, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 2의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 2, dist_list_start, dist_mat)))

k=3일 때 클러스터 0의 최단거리 방문 순서 : (0, 4, 2, 6, 3, 10)
k=3일 때 클러스터 1의 최단거리 방문 순서 : (9,)
k=3일 때 클러스터 2의 최단거리 방문 순서 : (8, 7, 5, 1)


In [21]:
print("K = 3")
geo_df_3_start = min_var_start_algorithm(geo_df_3_start, dist_list_start, dist_mat)

K = 3
Var : 1.8853e+07

변경 횟수 : 1
Var : 7.1664e+06

총 변경 횟수 : 1
최종 결과 Var : 7.1664e+06


In [22]:
print("K = 4")
geo_df_4_start = min_var_start_algorithm(geo_df_4_start, dist_list_start, dist_mat)

K = 4
Var : 2.0167e+07

변경 횟수 : 1
Var : 4.1288e+06

변경 횟수 : 2
Var : 3.4843e+06

변경 횟수 : 3
Var : 3.1796e+06

변경 횟수 : 4
Var : 2.1784e+06

변경 횟수 : 5
Var : 2.1308e+06

변경 횟수 : 6
Var : 1.3767e+06

총 변경 횟수 : 6
최종 결과 Var : 1.3767e+06


# Visualization

In [23]:
# 직선으로 시각화
def visual_start_line(geo_df_k_start, start_loc, dist_list_start, dist_mat):
    labels = geo_df_k_start['labels']
    m = folium.Map(location = [start_loc[1], start_loc[0]], zoom_start=14)

    # 각 위치 분류별로 색 다르게
    for i in range(geo_df_k_start.shape[0]):
        if (labels[i] == 0):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'blue')).add_to(m)
        elif (labels[i] == 1):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'red')).add_to(m)
        elif (labels[i] == 2):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'green')).add_to(m)
        elif (labels[i] == 3):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'purple')).add_to(m)
        else:
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'black')).add_to(m)
            
    # 시작 위치
    folium.CircleMarker(location=[start_loc[1], start_loc[0]],
                            radius=20,
                            color='red',
                            fill_color='red',
                            draggable = False).add_to(m)

    # 경로 표시
    colors = ['blue', 'red', 'green', 'purple']

    for i in np.unique(labels):
        orders = min_dist_start_order(geo_df_k_start, i, dist_list_start, dist_mat)
        
        location = []
        for order in orders:
            location.append([geo_df_k_start.loc[order, 'latitude'], geo_df_k_start.loc[order, 'longitude']])
        location.insert(0, [float(start_loc[1]), float(start_loc[0])])
        location.append([float(start_loc[1]), float(start_loc[0])])
        
        # print(location)

        folium.PolyLine(locations = location, tooltip = 'Polyline', color = colors[i]).add_to(m)

    return m

In [24]:
# K=3
visual_start_line(geo_df_3_start, start_loc, dist_list_start, dist_mat)

In [25]:
# K=4
visual_start_line(geo_df_4_start, start_loc, dist_list_start, dist_mat)

In [26]:
# driving 경로로 시각화
def visual_start_driving_line(geo_df_k_start, start_loc, dist_list_start, dist_mat):
    labels = geo_df_k_start['labels']
    m = folium.Map(location = [start_loc[1], start_loc[0]], zoom_start=14)

    # 각 위치 분류별로 색 다르게
    for i in range(geo_df_k_start.shape[0]):
        if (labels[i] == 0):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'blue')).add_to(m)
        elif (labels[i] == 1):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'red')).add_to(m)
        elif (labels[i] == 2):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'green')).add_to(m)
        elif (labels[i] == 3):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'purple')).add_to(m)
        else:
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'black')).add_to(m)
            
    # 중심 위치
    folium.CircleMarker(location=[start_loc[1], start_loc[0]],
                            radius=20,
                            color='red',
                            fill_color='red',
                            draggable = False).add_to(m)

    # 경로 표시
    colors = ['blue', 'red', 'green', 'purple']

    for i in np.unique(labels):
        orders = min_dist_start_order(geo_df_k_start, i, dist_list_start, dist_mat)
        
        location = []
        location.append([float(start_loc[1]), float(start_loc[0])])
        
        results = get_optimal_route(start = start_loc, goal = geo_list[orders[0]], option='traoptimal')
        for t in range(len(results['route']['traoptimal'][0]["path"])):
            location.append([results['route']['traoptimal'][0]["path"][t][1], results['route']['traoptimal'][0]["path"][t][0]])
        
        for j in range(len(orders)-1):
            location.append([geo_list[orders[j]][1], geo_list[orders[j]][0]])
            results = get_optimal_route(start = geo_list[orders[j]], goal = geo_list[orders[j+1]], option='traoptimal')
            for t in range(len(results['route']['traoptimal'][0]["path"])):
                location.append([results['route']['traoptimal'][0]["path"][t][1], results['route']['traoptimal'][0]["path"][t][0]])
        location.append([geo_list[orders[-1]][1], geo_list[orders[-1]][0]])
        
        results = get_optimal_route(start = geo_list[orders[-1]], goal = start_loc, option='traoptimal')
        for t in range(len(results['route']['traoptimal'][0]["path"])):
            location.append([results['route']['traoptimal'][0]["path"][t][1], results['route']['traoptimal'][0]["path"][t][0]])
        
        location.append([float(start_loc[1]), float(start_loc[0])])

        # print(location)

        folium.PolyLine(locations = location, tooltip = 'Polyline', color = colors[i]).add_to(m)

    return m

In [27]:
# K=3
visual_start_driving_line(geo_df_3_start, start_loc, dist_list_start, dist_mat)

In [28]:
# K=4
visual_start_driving_line(geo_df_4_start, start_loc, dist_list_start, dist_mat)